# Introduction to SQLite &
Selecting Sources from the Sloan Digital Sky Survey
========

#### Version 0.1

***
By AA Miller 2019 Mar 25



As noted earlier, there will be full lectures on databases over the remainder of this week.

This notebook provides a quick introduction to [`SQLite`](https://sqlite.org/index.html) a lightweight implementation of a Structured Query Language (SQL) database. One of the incredibly nice things about `SQLite` is the low overhead needed to set up a database (as you will see in a minute). We will take advantage of this low overhead to build a database later in the week.

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook

At the most basic level - databases *store your bytes*, and later *return those bytes* (or a subset of them) when queried. 

They provide a highly efficient means for filtering your bytes (there are many different strategies that the user can employ). 

The backend for most databases is the **Structured Query Language** or SQL, which is a standard declarative language. 

There are many different libraries that implement SQL: MySQL, PostgreSQL, Greenplum, Microsoft SQL server, IBM DB2, Oracle Database, etc. 

## Problem 1) Basic SQL Operations with SQLite

The most basic implementation is [`SQLite`](https://www.sqlite.org) a self-contained, SQL database engine. We will discuss `SQLite` further later in the week, but in brief - it is a nice stand alone package that works really well for small problems (such as the example that we are about to encounter). 

In [2]:
import sqlite3

Without diving too much into the weeds (we'll investigate this further later this week), we need to establish a [`connection`](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection) to the database. From the `connection` we create a [`cursor`](https://docs.python.org/2/library/sqlite3.html#sqlite3.Connection), which allows us to actually interact with the database. 

In [3]:
conn = sqlite3.connect("intro.db")
cur = conn.cursor()

And just like that - we have now created a new database `intro.db`, with which we can "store bytes" or later "retrieve bytes" once we have added some data to the database. 

*Aside* - note that unlike many SQL libraries, `SQLite` does not require a server and creates an actual database file on your hard drive. This improves portability, but also creates some downsides as well.

Now we need to create a table and insert some data. We will interact with the database via the [`execute()`](https://docs.python.org/2/library/sqlite3.html#sqlite3.Cursor.execute) method for the `cursor` object.

Recall that creating a table requires a specification of the table name, the columns in the table, and the data type for each column. Here's an example where I create a table to store info on my pets:

    cur.execute("""create table PetInfo(
                                        Name text, 
                                        Species text,
                                        Age tinyint,
                                        FavoriteFood text
                                        )""")

**Problem 1a**

Create a new table in the database called `DSFPstudents` with columns `Name`, `Institution`, and `Year`, where `Year` is the year in graduate school.

In [ ]:
cur.execute( # complete

In [4]:
cur.execute("""create table DSFPstudents(
                                    Name text, 
                                    Institution text,
                                    Year tinyint
                                    )""")

Once a table is created, we can use the database to store bytes. If I were to populate my `PetInfo` table I would do the following:

    cur.execute("""insert into PetInfo(Name, Species, Age, FavoriteFood) 
                                values ("Rocky", "Dog", 12, "Bo-Nana")""")
    cur.execute("""insert into PetInfo(Name, Species, Age, FavoriteFood) 
                                values ("100 Emoji-Flames Emoji", "Red Panda", 2, "bamboo leaves")""")

*Note* - column names do not need to be explicitly specified, but for clarity this is always preferred.

**Problem 1b**

Insert data for yourself, and the two people sitting next to you into the database.

In [ ]:
cur.execute( # complete

In [5]:
cur.execute("""insert into DSFPstudents(Name, Institution, Year) 
                            values ("Adam Miller", "Northwestern", 13)""")
cur.execute("""insert into DSFPstudents(Name, Institution, Year) 
                            values ("Lucianne Walkowicz", "Adler", 14)""")

Now that we have bytes in the database, we can retrieve those bytes with one (or several) queries. There are 3 basic building blocks to a query:

    SELECT...
    FROM...
    WHERE...
    
Where `SELECT` specifies the information we want to retrieve from the database, `FROM` specifies the tables being queried in the database, and `WHERE` specifies the conditions for the query. 

**Problem 1c**

Select the institutions for all students in the `DSFPstudents` table who have been in grad school for more than 2 years.

*Hint* - to display the results of your query run `cur.fetchall()`.

In [ ]:
cur.execute( # complete
cur.fetchall()

In [6]:
cur.execute("""select Institution from DSFPstudents where year > 2""")
cur.fetchall()

[('Northwestern',), ('Adler',)]

In closing this brief introduction to databases, note that good databases follow the 4 ACID properties:

1. Atomicity
2. Consistency
3. Isolation
4. Durability

In closing this brief introduction to databases, note that good databases follow the 4 ACID properties:

1. Atomicity - all parts of transaction succeed, or rollback state of database
2. Consistency
3. Isolation
4. Durability

In closing this brief introduction to databases, note that good databases follow the 4 ACID properties:

1. Atomicity - all parts of transaction succeed, or rollback state of database
2. Consistency - data always meets validation rules
3. Isolation
4. Durability

In closing this brief introduction to databases, note that good databases follow the 4 ACID properties:

1. Atomicity - all parts of transaction succeed, or rollback state of database
2. Consistency - data always meets validation rules
3. Isolation - no interference across transactions (even if concurrent)
4. Durability

In closing this brief introduction to databases, note that good databases follow the 4 ACID properties:

1. Atomicity - all parts of transaction succeed, or rollback state of database
2. Consistency - data always meets validation rules
3. Isolation - no interference across transactions (even if concurrent)
4. Durability - a committed transaction remains committed (even if there's a power outage, etc)

## Problem 2) Complex Queries with SDSS

Above we looked at the most basic operations possible with a database (recall - databases are unnecessary, and possibly cumbersome, with small data sets). A typical database consists of many tables, and these tables may be joined together to unlock complex questions for the data. 

As a reminder on (some of) this functionality, we are now going to go through some problems using the SDSS database. The full [SDSS schema](http://skyserver.sdss.org/dr13/en/help/browser/browser.aspx) explains all of the tables, columns, views and functions for querying the database. We will keep things relatively simple in that regard.

In [7]:
# you may need to run conda install -c astropy astroquery

from astroquery.sdss import SDSS

/Users/adamamiller/miniconda3/envs/py36/lib/python3.6/site-packages/astroquery/sdss/__init__.py:29: UserWarning: Experimental: SDSS has not yet been refactored to have its API match the rest of astroquery (but it's nearly there).
  warnings.warn("Experimental: SDSS has not yet been refactored to have its API "


[`astroquery`](http://astroquery.readthedocs.io/en/latest/) enables seemless connections to the SDSS database via the Python shell.

**Problem 2a**

Select 20 random sources from the [`PhotoObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=PhotoObjAll) table and return all columns in the table. 

*Hint* - while this would normally be accomplished by starting the query `select limit 20 ...`, SDSS CasJobs uses Microsoft's SQL Server, which adopts `select top 20 ...` to accomplish an identical result.

In [ ]:
SDSS.query_sql( # complete

In [8]:
SDSS.query_sql("""select top 20 * from PhotoObjAll""")

objID,skyVersion,run,rerun,camcol,field,obj,mode,nChild,type,clean,probPSF,insideMask,flags,rowc,rowcErr,colc,colcErr,rowv,rowvErr,colv,colvErr,rowc_u,rowc_g,rowc_r,rowc_i,rowc_z,rowcErr_u,rowcErr_g,rowcErr_r,rowcErr_i,rowcErr_z,colc_u,colc_g,colc_r,colc_i,colc_z,colcErr_u,colcErr_g,colcErr_r,colcErr_i,colcErr_z,sky_u,sky_g,sky_r,sky_i,sky_z,skyIvar_u,skyIvar_g,skyIvar_r,skyIvar_i,skyIvar_z,psfMag_u,psfMag_g,psfMag_r,psfMag_i,psfMag_z,psfMagErr_u,psfMagErr_g,psfMagErr_r,psfMagErr_i,psfMagErr_z,fiberMag_u,fiberMag_g,fiberMag_r,fiberMag_i,fiberMag_z,fiberMagErr_u,fiberMagErr_g,fiberMagErr_r,fiberMagErr_i,fiberMagErr_z,fiber2Mag_u,fiber2Mag_g,fiber2Mag_r,fiber2Mag_i,fiber2Mag_z,fiber2MagErr_u,fiber2MagErr_g,fiber2MagErr_r,fiber2MagErr_i,fiber2MagErr_z,petroMag_u,petroMag_g,petroMag_r,petroMag_i,petroMag_z,petroMagErr_u,petroMagErr_g,petroMagErr_r,petroMagErr_i,petroMagErr_z,psfFlux_u,psfFlux_g,psfFlux_r,psfFlux_i,psfFlux_z,psfFluxIvar_u,psfFluxIvar_g,psfFluxIvar_r,psfFluxIvar_i,psfFluxIvar_z,fiberFlux_u,fiberFlux_g,fiberFlux_r,fiberFlux_i,fiberFlux_z,fiberFluxIvar_u,fiberFluxIvar_g,fiberFluxIvar_r,fiberFluxIvar_i,fiberFluxIvar_z,fiber2Flux_u,fiber2Flux_g,fiber2Flux_r,fiber2Flux_i,fiber2Flux_z,fiber2FluxIvar_u,fiber2FluxIvar_g,fiber2FluxIvar_r,fiber2FluxIvar_i,fiber2FluxIvar_z,petroFlux_u,petroFlux_g,petroFlux_r,petroFlux_i,petroFlux_z,petroFluxIvar_u,petroFluxIvar_g,petroFluxIvar_r,petroFluxIvar_i,petroFluxIvar_z,petroRad_u,petroRad_g,petroRad_r,petroRad_i,petroRad_z,petroRadErr_u,petroRadErr_g,petroRadErr_r,petroRadErr_i,petroRadErr_z,petroR50_u,petroR50_g,petroR50_r,petroR50_i,petroR50_z,petroR50Err_u,petroR50Err_g,petroR50Err_r,petroR50Err_i,petroR50Err_z,petroR90_u,petroR90_g,petroR90_r,petroR90_i,petroR90_z,petroR90Err_u,petroR90Err_g,petroR90Err_r,petroR90Err_i,petroR90Err_z,q_u,q_g,q_r,q_i,q_z,qErr_u,qErr_g,qErr_r,qErr_i,qErr_z,u_u,u_g,u_r,u_i,u_z,uErr_u,uErr_g,uErr_r,uErr_i,uErr_z,mE1_u,mE1_g,mE1_r,mE1_i,mE1_z,mE2_u,mE2_g,mE2_r,mE2_i,mE2_z,mE1E1Err_u,mE1E1Err_g,mE1E1Err_r,mE1E1Err_i,mE1E1Err_z,mE1E2Err_u,mE1E2Err_g,mE1E2Err_r,mE1E2Err_i,mE1E2Err_z,mE2E2Err_u,mE2E2Err_g,mE2E2Err_r,mE2E2Err_i,mE2E2Err_z,mRrCc_u,mRrCc_g,mRrCc_r,mRrCc_i,mRrCc_z,mRrCcErr_u,mRrCcErr_g,mRrCcErr_r,mRrCcErr_i,mRrCcErr_z,mCr4_u,mCr4_g,mCr4_r,mCr4_i,mCr4_z,mE1PSF_u,mE1PSF_g,mE1PSF_r,mE1PSF_i,mE1PSF_z,mE2PSF_u,mE2PSF_g,mE2PSF_r,mE2PSF_i,mE2PSF_z,mRrCcPSF_u,mRrCcPSF_g,mRrCcPSF_r,mRrCcPSF_i,mRrCcPSF_z,mCr4PSF_u,mCr4PSF_g,mCr4PSF_r,mCr4PSF_i,mCr4PSF_z,deVRad_u,deVRad_g,deVRad_r,deVRad_i,deVRad_z,deVRadErr_u,deVRadErr_g,deVRadErr_r,deVRadErr_i,deVRadErr_z,deVAB_u,deVAB_g,deVAB_r,deVAB_i,deVAB_z,deVABErr_u,deVABErr_g,deVABErr_r,deVABErr_i,deVABErr_z,deVPhi_u,deVPhi_g,deVPhi_r,deVPhi_i,deVPhi_z,deVMag_u,deVMag_g,deVMag_r,deVMag_i,deVMag_z,deVMagErr_u,deVMagErr_g,deVMagErr_r,deVMagErr_i,deVMagErr_z,deVFlux_u,deVFlux_g,deVFlux_r,deVFlux_i,deVFlux_z,deVFluxIvar_u,deVFluxIvar_g,deVFluxIvar_r,deVFluxIvar_i,deVFluxIvar_z,expRad_u,expRad_g,expRad_r,expRad_i,expRad_z,expRadErr_u,expRadErr_g,expRadErr_r,expRadErr_i,expRadErr_z,expAB_u,expAB_g,expAB_r,expAB_i,expAB_z,expABErr_u,expABErr_g,expABErr_r,expABErr_i,expABErr_z,expPhi_u,expPhi_g,expPhi_r,expPhi_i,expPhi_z,expMag_u,expMag_g,expMag_r,expMag_i,expMag_z,expMagErr_u,expMagErr_g,expMagErr_r,expMagErr_i,expMagErr_z,modelMag_u,modelMag_g,modelMag_r,modelMag_i,modelMag_z,modelMagErr_u,modelMagErr_g,modelMagErr_r,modelMagErr_i,modelMagErr_z,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z,cModelMagErr_u,cModelMagErr_g,cModelMagErr_r,cModelMagErr_i,cModelMagErr_z,expFlux_u,expFlux_g,expFlux_r,expFlux_i,expFlux_z,expFluxIvar_u,expFluxIvar_g,expFluxIvar_r,expFluxIvar_i,expFluxIvar_z,modelFlux_u,modelFlux_g,modelFlux_r,modelFlux_i,modelFlux_z,modelFluxIvar_u,modelFluxIvar_g,modelFluxIvar_r,modelFluxIvar_i,modelFluxIvar_z,cModelFlux_u,cModelFlux_g,cModelFlux_r,cModelFlux_i,cModelFlux_z,cModelFluxIvar_u,cModelFluxIvar_g,cModelFluxIvar_r,cModelFluxIvar_i,cModelFluxIvar_z,aperFlux7_u,aperFlux7_g,aperFlux7_r,aperF

That's more columns than we will likely ever need. Instead, let's focus on `objID`, a unique identifier, `cModelMag_u`, `cModelMag_g`, `cModelMag_r`, `cModelMag_i`, and `cModelMag_z`, the source magnitude in $u', g', r', i', z'$, respectively.

We will now introduce the concept of joining two tables. 

The most common operation is known as an `inner join` (which is often referred to as just `join`). An `inner join` returns records that have matching sources in both tables in the join. 

Less, but nevertheless still powerful, is the `outer join`. An outer join returns *all* records in either table, with `NULL` values for columns in a table in which the record does not exist.

Specialized versions of the `outer join` include the `left join` and `right join`, whereby *all* records in either the left or right table, respectively, are returned along with their counterparts. 

**Problem 2b**

Select `objid` and $u'g'r'i'z'$ from `PhotoObjAll` and the corresponding `class` from [`specObjAll`](https://skyserver.sdss.org/dr13/en/help/docs/tabledesc.aspx?name=SpecObjAll) for 20 random sources. 

There are multiple columns you could use to join the tables, in this case match `objid` to `bestobjid` from `specObjAll` and use an `inner join`.

In [ ]:
SDSS.query_sql( # complete

In [9]:
SDSS.query_sql("""select top 20 objid, cModelMag_u, cModelMag_g, cModelMag_r, cModelMag_i, cModelMag_z, 
                         class
                  from photoobjall p 
                      inner join specobjall s on p.objid = s.bestobjid""")

/Users/adamamiller/miniconda3/envs/py36/lib/python3.6/site-packages/astroquery/sdss/core.py:857: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


objid,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z,class
int64,float64,float64,float64,float64,float64,bytes6
1237645879551000764,20.18099,20.7678,19.37491,18.44864,18.55745,GALAXY
1237645879551066262,19.52121,18.12321,17.48715,17.12531,16.96935,GALAXY
1237645879562862699,20.13848,19.28518,19.03111,18.94487,18.73979,STAR
1237645879562928144,22.76108,20.03992,18.60326,18.08318,17.80411,GALAXY
1237645879562928258,21.18919,20.25929,19.24559,18.79132,18.41072,GALAXY
1237645879562928805,21.55341,22.13135,21.31751,19.50049,18.87878,GALAXY
1237645879573873373,19.90373,21.73595,20.20402,19.485,19.05127,GALAXY
1237645879577477252,21.55177,18.92065,17.75269,17.26517,16.88253,GALAXY
1237645879577477252,21.55177,18.92065,17.75269,17.26517,16.88253,GALAXY


**Problem 2c**

Perform an identical query to the one above, but this time use a `left outer join` (or `left join`).

How do your results compare to the previous query?

In [10]:
SDSS.query_sql("""select top 20 objid, cModelMag_u, cModelMag_g, cModelMag_r, cModelMag_i, cModelMag_z, 
                         class
                  from photoobjall p 
                      left outer join specobjall s on p.objid = s.bestobjid""")

objid,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z,class
int64,float64,float64,float64,float64,float64,bool
1237645876861272065,13.07936,12.57809,12.34195,11.81598,11.65377,False
1237645876861272066,13.08528,13.02686,12.85163,11.88167,11.64233,False
1237645876861272067,19.71465,13.86158,14.17228,15.64151,14.23906,False
1237645876861272068,22.64303,15.89897,21.31149,20.91684,20.02921,False
1237645876861272069,13.1023,12.64387,12.39441,11.95594,11.64456,False
1237645876861272070,13.15823,12.88329,12.59817,12.43937,11.76545,False
1237645876861272071,24.35513,15.19132,14.50776,14.00838,22.82691,False
1237645876861272072,24.6347,16.45857,24.80208,14.57438,22.82694,False
1237645876861272073,24.03423,28.21009,24.80906,17.96751,25.42343,False


**Problem 2d**

This time use a `right outer join` (or `right join`).

How do your results compare to the previous query?

In [11]:
SDSS.query_sql("""select top 20 objid, cModelMag_u, cModelMag_g, cModelMag_r, cModelMag_i, cModelMag_z, 
                         class
                  from photoobjall p 
                      right outer join specobjall s on s.bestobjid = p.objid
                """)

/Users/adamamiller/miniconda3/envs/py36/lib/python3.6/site-packages/astroquery/sdss/core.py:857: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


objid,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z,class
bool,bool,bool,bool,bool,bool,bytes6
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY
False,False,False,False,False,False,GALAXY


## Challenge Problem

To close the notebook we will perform a nested query. In brief, the idea is to join the results of one query with a separate query.

Here, we are going to attempt to identify bright AGN that don't have SDSS spectra. To do so we will need the `photoObjAll` table, the `specObjAll` table, and the `rosat` table, which includes all cross matches between SDSS sources and X-ray sources detected by the [Rosat satellite](https://heasarc.gsfc.nasa.gov/docs/rosat/rosat3.html).

Create a nested query that selects all *Rosat* sources that don't have SDSS spectra with `cModelFlux_u + cModelFlux_g + cModelFlux_r + cModelFlux_i + cModelFlux_z > 10000` (this flux contraint ensures the source is bright without making any cuts on color) and `type = 3`, this last constraint means the source is extended in SDSS images. 

*Hint* - you may run into timeout issues in which case you should run the query on CasJobs.

In [12]:
SDSS.query_sql("""select rm.*
from 
(select r.objid, r.sourcename, r.ra, r.dec, r.cps, r.hr1, r.hr2, cModelMag_u, cModelMag_g, cModelMag_r, cModelMag_i, cModelMag_z
from photoobjall p join rosat r on p.objid = r.objid
  where (cModelFlux_u + cModelFlux_g + cModelFlux_r + cModelFlux_i + cModelFlux_z > 10000)
    and p.type = 3) as rm
  left join specobjall p on rm.objid = p.bestobjid
  where p.bestobjid is null 
                """)

/Users/adamamiller/miniconda3/envs/py36/lib/python3.6/site-packages/astroquery/sdss/core.py:857: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  comments='#'))


objid,sourcename,ra,dec,cps,hr1,hr2,cModelMag_u,cModelMag_g,cModelMag_r,cModelMag_i,cModelMag_z
int64,bytes16,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1237672132540039389,J190652.7+221022,286.7196,22.17278,0.02739,0.1,-0.52,11.43623,11.16234,10.99726,10.28161,9.645877
1237680505044009377,J225630.4+335507,344.12668,33.91861,0.03514,1.0,-0.09,13.90497,13.16328,13.27648,12.72426,12.31295
1237679341636616241,J012500.9-055646,21.25375,-5.94625,0.03956,0.57,0.51,10.91721,10.90554,10.37871,10.28698,9.828402
1237658611448021052,J121211.5+484859,183.04791,48.81653,0.07175,-0.14,0.38,18.49315,16.31862,14.29825,12.89009,11.96395
1237654641284022344,J144201.8+611543,220.50749,61.26208,0.04803,-0.38,-0.14,24.4289,12.57266,11.36399,11.00803,12.00601
1237666634462527495,J001745.3-010255,4.43875,-1.04861,0.09727,-0.05,-0.12,12.87801,13.36069,12.8068,12.46747,11.63604
1237671686937641099,J114555.0+720550,176.47917,72.09722,0.02236,-0.49,0.23,24.63464,25.11437,14.04865,12.79556,22.82689
1237662225162371182,J160248.3+252031,240.70123,25.34208,0.1967,0.1,0.17,14.38885,14.38303,13.57788,13.3331,12.67912
1237674285401244515,J062937.2+082930,97.405,8.49181,0.08809,-0.43,-0.66,11.98673,11.68047,11.40246,11.11554,10.38265
